# Generar y subir datos IoT a S3

Este notebook genera un JSON con datos IoT sintéticos y lo sube a S3 en `data/sensores/...`.

## 1) Instalar dependencias

Ejecuta esta celda para instalar automáticamente lo de `requirements.txt`.

In [15]:
from pathlib import Path

req = Path("requirements.txt")
if not req.exists():
    req = Path("../requirements.txt")
if not req.exists():
    raise FileNotFoundError("No se encontró requirements.txt")

%pip install -r {req}

Note: you may need to restart the kernel to use updated packages.


## 2) Configuración

Rellena estos valores con tu bucket y región.

In [16]:
AWS_REGION = "us-east-1"
S3_BUCKET = "ra2-sbd-bertobm-20260221-8473"
S3_KEY = "data/sensores/iabd13_sensores.json"
ROWS = 200
SENSORS = 5

## 3) Generar datos IoT

In [17]:
from datetime import datetime, timedelta, timezone
import json
import random
from pathlib import Path

def sensor_state_from_values(temperature_c: float, co2_ppm: float) -> str:
    if temperature_c >= 30 or co2_ppm >= 900:
        return "FAIL"
    if temperature_c >= 26 or co2_ppm >= 700:
        return "WARN"
    return "OK"

def generate_records(rows: int, sensors: int):
    base_time = datetime.now(timezone.utc) - timedelta(minutes=5 * rows)
    sensor_ids = [f"SEN-{i:03d}" for i in range(1, sensors + 1)]
    base_locations = [
        (40.4168, -3.7038),
        (41.3874, 2.1686),
        (39.4699, -0.3763),
        (37.3891, -5.9845),
        (43.2630, -2.9350),
    ]

    records = []
    for i in range(rows):
        sensor_id = random.choice(sensor_ids)
        lat0, lon0 = random.choice(base_locations)
        temperature_c = round(random.uniform(18.0, 33.0), 2)
        co2_ppm = round(random.uniform(420.0, 1100.0), 2)
        state = sensor_state_from_values(temperature_c, co2_ppm)

        records.append({
            "timestamp": (base_time + timedelta(minutes=5 * i)).isoformat(),
            "sensor_id": sensor_id,
            "sensor_state": state,
            "temperature_c": temperature_c,
            "co2_ppm": co2_ppm,
            "lat": round(lat0 + random.uniform(-0.02, 0.02), 6),
            "lon": round(lon0 + random.uniform(-0.02, 0.02), 6),
        })
    return records

records = generate_records(ROWS, SENSORS)
len(records), records[0]

(200,
 {'timestamp': '2026-02-21T07:12:34.971512+00:00',
  'sensor_id': 'SEN-004',
  'sensor_state': 'WARN',
  'temperature_c': 29.08,
  'co2_ppm': 513.54,
  'lat': 37.37651,
  'lon': -5.98906})

## 4) Guardar JSON local (opcional)

In [18]:
local_out = Path("./data/iabd13_sensores.json")
local_out.write_text(json.dumps(records, ensure_ascii=True, indent=2), encoding="utf-8")
local_out

PosixPath('data/iabd13_sensores.json')

## 5) Subir a S3

In [ ]:
import os
import boto3

for k in ["AWS_ACCESS_KEY_ID", "AWS_SECRET_ACCESS_KEY", "AWS_SESSION_TOKEN", "AWS_PROFILE", "AWS_DEFAULT_PROFILE"]:
    os.environ.pop(k, None)

session = boto3.Session(profile_name="awsacademy", region_name=AWS_REGION)
print(session.client("sts").get_caller_identity())

s3 = session.client("s3")
body = json.dumps(records, ensure_ascii=True, indent=2).encode("utf-8")
s3.put_object(Bucket=S3_BUCKET, Key=S3_KEY, Body=body, ContentType="application/json")
print(f"Subido a: s3://{S3_BUCKET}/{S3_KEY}")

{'UserId': 'AROAULJXNNU7ICVAJEMC7:user4587692=Alberto', 'Account': '299153321278', 'Arn': 'arn:aws:sts::299153321278:assumed-role/voclabs/user4587692=Alberto', 'ResponseMetadata': {'RequestId': '3bfca045-8336-4f02-8717-56243c351060', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '3bfca045-8336-4f02-8717-56243c351060', 'x-amz-sts-extended-request-id': 'MTp1cy1lYXN0LTE6UzoxNzcxNzE3OTU1NDQ5OlI6SExLQ000cnM=', 'content-type': 'text/xml', 'content-length': '452', 'date': 'Sat, 21 Feb 2026 23:52:35 GMT'}, 'RetryAttempts': 0}}
Subido a: s3://ra2-sbd-bertobm-20260221-8473/data/sensores/iabd13_sensores.json
